#For training

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install zipfile36
!pip install git+https://github.com/mr7495/RetinaNet

  Cloning https://github.com/mr7495/RetinaNet to /tmp/pip-req-build-g2rtlbs7
  Running command git clone -q https://github.com/mr7495/RetinaNet /tmp/pip-req-build-g2rtlbs7
  Running command git submodule update --init --recursive -q
  Created wheel for keras-retinanet: filename=keras_retinanet-0.5.1-cp37-cp37m-linux_x86_64.whl size=181933 sha256=74823bd8bb159dc6d7e4db46697a6c4282ab6817cf7efb6843438f2fe7c9ada9
  Stored in directory: /tmp/pip-ephem-wheel-cache-7jg7mkyq/wheels/08/6d/f5/38473102b36a5975e02e8f339fbf85bc4d1b1c7c80dc68a595
  Created wheel for keras-resnet: filename=keras_resnet-0.1.0-py2.py3-none-any.whl size=13346 sha256=01604df6cf26854e47c0d8b5d931b88536151cd6416f79d2ea4d943a4a147b36
  Stored in directory: /root/.cache/pip/wheels/80/dd/ac/842235b63dddac12faa4b48ebe58b8944e8c2e57c2e38dddb6
Successfully built keras-retinanet keras-resnet


In [ ]:
import keras
import numpy as np
import cv2
import os
import random
import shutil
import pandas as pd
import csv
import zipfile
from keras import optimizers
from keras.models import Sequential,Model
from keras.layers import Dropout, Flatten, Dense,Input
from keras.applications.resnet_v2 import ResNet50V2
from keras.applications.xception import Xception
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.callbacks import ModelCheckpoint
from keras.applications.imagenet_utils import preprocess_input
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.initializers import RandomNormal
import keras.backend as k
from sklearn.utils import shuffle
import io
from PIL import Image as pil_image
from keras_retinanet import layers
import keras.backend as k
import keras_retinanet

In [ ]:
archive = zipfile.ZipFile('/content/drive/MyDrive/Train&Validation.zip') 
for file in archive.namelist():
     archive.extract(file, './data') #extract all image to folder data for training

In [ ]:
#image data preprocessing
fold_num = 2
train_datagen = ImageDataGenerator(horizontal_flip = True, vertical_flip = True, zoom_range = 0.05, rotation_range = 360, width_shift_range = 0.05, height_shift_range = 0.05, shear_range = 0.05)
test_datagen = ImageDataGenerator()
train_df = pd.read_csv('/content/drive/MyDrive/CSV/train{}.csv'.format(fold_num)) #read train csv file
validation_df = pd.read_csv('/content/drive/MyDrive/CSV/validation{}.csv'.format(fold_num)) #read validation csv file (Validation in the training process)
train_df = shuffle(train_df) #shuffle the train data
test_df = pd.read_csv('/content/drive/MyDrive/CSV/test{}.csv'.format(fold_num)) #read test csv file (For evaluating the final version of the trained network)

In [ ]:
shape = (512, 512, 1) #shape of the dataset images (in TIFF format)

In [ ]:
train_generator = train_datagen.flow_from_dataframe(dataframe = train_df, directory='data', x_col = "filename", y_col = "class", target_size = (512, 512), batch_size = 14,
                                                    class_mode = 'categorical', color_mode = "grayscale", shuffle = True)
validation_generator = test_datagen.flow_from_dataframe(dataframe = validation_df, directory = 'data', x_col = "filename", y_col = "class", target_size = (512, 512), batch_size = 10,
                                                        class_mode = 'categorical', color_mode = "grayscale", shuffle=True)
test_generator = test_datagen.flow_from_dataframe(dataframe = test_df, directory = 'data', x_col = "filename", y_col = "class", target_size = (512, 512), batch_size = 10,
                                                  class_mode = 'categorical', color_mode = "grayscale", shuffle = True)

Found 3715 validated image filenames belonging to 2 classes.
Found 915 validated image filenames belonging to 2 classes.
Found 8343 validated image filenames belonging to 2 classes.


In [ ]:
from tensorflow.python.keras.callbacks import EarlyStopping

k.clear_session() #clear keras backend 
try:
  os.mkdir('models') #create folder for saving the model
except:
  pass
full_name = 'ResNet50V2-FPN-fold{}'.format(fold_num)
classes_number = 2 #normal and COVID-19
input_tensor = Input(shape=shape)
weight_model = ResNet50V2(weights='imagenet', include_top=False) #load ResNet50V2 ImageNet pre-trained weights
weight_model.save_weights('weights.h5') #save the weights
base_model = ResNet50V2(weights=None, include_top=False, input_tensor=input_tensor) #load the ResNet50V2 model without weights
base_model.load_weights('weights.h5',skip_mismatch=True, by_name=True) #load the ImageNet weights on the ResNet50V2 model except the first layer(because the first layer has one channel in our case)
#create FPN
#https://github.com/fizyr/keras-retinanet/blob/master/keras_retinanet/models/retinanet.py
feature_size = 256 #Set the feature channels of the FPN
layer_names = ["conv4_block1_preact_relu", "conv5_block1_preact_relu", "post_relu"] #Layers of ResNet50V2 with different scale features 
layer_outputs = [base_model.get_layer(name).output for name in layer_names]
C3, C4, C5 = layer_outputs #Features of different scales, extracted from ResNet50V2
P5 = keras.layers.Conv2D(feature_size, kernel_size=1, strides=1, padding='same', name='C5_reduced')(C5)
P5_upsampled = layers.UpsampleLike(name='P5_upsampled')([P5, C4])
P5 = keras.layers.Conv2D(feature_size, kernel_size=3, strides=1, padding='same', name='P5')(P5)
#Concatenate P5 elementwise to C4
P4 = keras.layers.Conv2D(feature_size, kernel_size=1, strides=1, padding='same', name='C4_reduced')(C4)
P4 = keras.layers.Concatenate(axis=3)([P5_upsampled, P4])
P4_upsampled = layers.UpsampleLike(name='P4_upsampled')([P4, C3])
P4 = keras.layers.Conv2D(feature_size, kernel_size=3, strides=1, name='P4')(P4)
#Concatenate P4 elementwise to C3
P3 = keras.layers.Conv2D(feature_size, kernel_size=1, strides=1, padding='same', name='C3_reduced')(C3)
P3 = keras.layers.Concatenate(axis=3)([P4_upsampled, P3])
P3 = keras.layers.Conv2D(feature_size, kernel_size=3, strides=1, name='P3')(P3)
#P6 is obtained via a 3x3 stride-2 conv on C5
P6 = keras.layers.Conv2D(feature_size, kernel_size=3, strides=2, padding='same', name='P6')(C5)
#P7 is computed by applying ReLU followed by a 3x3 stride-2 conv on P6
P7 = keras.layers.Activation('relu', name='C6_relu')(P6)
P7 = keras.layers.Conv2D(feature_size, kernel_size=3, strides=2, padding='same', name='P7')(P7)
#Run classification for each of the generated features from the pyramid
feature1 = Flatten()(P3)
dp1 = Dropout(0.5)(feature1)
preds1 = Dense(2, activation='relu',kernel_initializer=RandomNormal(mean=0.0, stddev=0.001))(dp1)

feature2 = Flatten()(P4)
dp2 = Dropout(0.5)(feature2)
preds2 = Dense(2, activation='relu',kernel_initializer=RandomNormal(mean=0.0, stddev=0.001))(dp2)

feature3 = Flatten()(P5)
dp3= Dropout(0.5)(feature3)
preds3 = Dense(2, activation='relu',kernel_initializer=RandomNormal(mean=0.0, stddev=0.001))(dp3)

feature4 = Flatten()(P6)
dp4 = Dropout(0.5)(feature4)
preds4 = Dense(2, activation='relu',kernel_initializer=RandomNormal(mean=0.0, stddev=0.001))(dp4)

feature5 = Flatten()(P7)
dp5 = Dropout(0.5)(feature5)
preds5 = Dense(2, activation='relu',kernel_initializer=RandomNormal(mean=0.0, stddev=0.001))(dp5)

concat = keras.layers.Concatenate(axis=1)([preds1, preds2, preds3, preds4, preds5]) #Concatenate the predictions(Classification results) of each of the pyramid features 
out = keras.layers.Dense(2, activation='softmax', kernel_initializer=RandomNormal(mean=0.0, stddev=0.001))(concat) #Final Classification

model = Model(inputs=base_model.input, outputs=out) 
for layer in model.layers:
  layer.trainable = True
model.compile(optimizer=optimizers.Nadam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
filepath = "models/%s-{epoch:02d}-{val_accuracy:.4f}.hdf5"%full_name #path to save the model
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', save_best_only=True, mode='max') #save the best validation accuracy
#model.fit_generator(train_generator, epochs=20, validation_data=validation_generator, shuffle=True, callbacks=[checkpoint]) 
early_stopper = EarlyStopping(monitor = 'val_loss', patience = 3)
model.fit(train_generator, epochs=20, validation_data=validation_generator, shuffle=True, callbacks=[checkpoint, early_stopper]) 

94674944/94668760 [==============================] - 1s 0us/step
Epoch 1/20
266/266 [==============================] - 496s 2s/step - loss: 0.4274 - accuracy: 0.7902 - val_loss: 0.1847 - val_accuracy: 0.9399
Epoch 2/20
266/266 [==============================] - 430s 2s/step - loss: 0.1291 - accuracy: 0.9621 - val_loss: 0.2982 - val_accuracy: 0.9268
Epoch 3/20
266/266 [==============================] - 429s 2s/step - loss: 0.1132 - accuracy: 0.9599 - val_loss: 0.2970 - val_accuracy: 0.9060
Epoch 4/20
266/266 [==============================] - 429s 2s/step - loss: 0.0817 - accuracy: 0.9703 - val_loss: 0.1432 - val_accuracy: 0.9563
Epoch 5/20
266/266 [==============================] - 429s 2s/step - loss: 0.0780 - accuracy: 0.9771 - val_loss: 0.1059 - val_accuracy: 0.9716
Epoch 6/20
266/266 [==============================] - 429s 2s/step - loss: 0.0494 - accuracy: 0.9825 - val_loss: 0.2198 - val_accuracy: 0.9530
Epoch 7/20
266/266 [==============================] - 429s 2s/step - loss: 0.

In [ ]:
!cp /content/models/ResNet50V2-FPN-fold2-05-0.9716.hdf5 /content/drive/MyDrive/backup_models/ResNet50V2_FPN_Fold2

In [ ]:
!rm -rf /content/models